In [2]:
# --------------------------------------------
#                   TEST LCU
# --------------------------------------------
from src import lcu
lcu = lcu.LCU()

instr_list = ["NOP", "EXIT", "SADD R1, ZERO, LAST", "SADD R1, SRF(3), LAST", "SADDI R1, ONE, 7", "SSUB SRF(4), SRF(4), SRF(4)", "JUMP ONE, 7", "BGEPD ONE, ZERO, 5"]

for instr in instr_list:
    srf_read, srf_str, word = lcu.asmToHex(instr)
    print(instr + ": " + str(srf_read) + ", " + str(srf_str) + ", " + word.get_word_in_hex())

NOP: -1, -1, 0x0
EXIT: -1, -1, 0x1c00
SADD R1, ZERO, LAST: -1, -1, 0xb8340
SADD R1, SRF(3), LAST: 3, -1, 0xb0340
SADDI R1, ONE, 7: -1, -1, 0xfc347
SSUB SRF(4), SRF(4), SRF(4): 4, 4, 0x90400
JUMP ONE, 7: -1, -1, 0xfda07
BGEPD ONE, ZERO, 5: -1, -1, 0xdd605


In [3]:
# --------------------------------------------
#                   TEST LSU
# --------------------------------------------
from src import lsu
lsu = lsu.LSU()

instr_list = ["SADD R0, ONE, ONE/LD.VWR VWR_A", "SADD R0, SRF(5), ONE/SH.IL.UP", "SADD SRF(5), SRF(5), ONE/LD.VWR SRF"]

for instr in instr_list:
    srf_read, srf_str, word = lsu.asmToHex(instr)
    print(instr + ": " + str(srf_read) + ", " + str(srf_str) + ", " + word.get_word_in_hex())

SADD R0, ONE, ONE/LD.VWR VWR_A: -1, -1, 0x45538
SADD R0, SRF(5), ONE/SH.IL.UP: 5, -1, 0xc4538
SADD SRF(5), SRF(5), ONE/LD.VWR SRF: 5, 5, 0x5c530


In [4]:
# --------------------------------------------
#                   TEST RCs
# --------------------------------------------
from src import rc
rc = rc.RC()

instr_list = ["NOP", "SADD VWR_A, VWR_A, VWR_B", "SADD VWR_A, SRF(3), VWR_B", "LOR R0, RCB, MIN_INT", "MUL.FP R0, RCB, MIN_INT"]
# DIV.FP R0, RCB, MIN_INT") Error: float point div not supported
for instr in instr_list:
    srf_read, srf_str, vwr_str, word = rc.asmToHex(instr)
    print(instr + ": " + str(srf_read) + ", " + str(srf_str) + ", " + word.get_word_in_hex())

NOP: -1, -1, 0x0
SADD VWR_A, VWR_A, VWR_B: -1, -1, 0x420
SADD VWR_A, SRF(3), VWR_B: 3, -1, 0xc420
LOR R0, RCB, MIN_INT: -1, -1, 0x1f522
MUL.FP R0, RCB, MIN_INT: -1, -1, 0x1f5a2


In [5]:
# --------------------------------------------
#                   TEST MXCU
# --------------------------------------------
from src import mxcu
mxcu = mxcu.MXCU()

instr_list = ["NOP", "SADD R1, ONE, LAST", "LOR R1, ONE, SRF(3)"]
srf_sel = -1
srf_we = 0
alu_srf_write = 0
vwr_row_we = [0,0,0,0]
vwr_sel = 0
for instr in instr_list:
    word = mxcu.asmToHex(instr, srf_sel, srf_we, alu_srf_write, vwr_row_we, vwr_sel)
    print(instr + ": " + word.get_word_in_hex())

NOP: 0x0
SADD R1, ONE, LAST: 0x5699000
LOR R1, ONE, SRF(3): 0x54690c0


In [6]:
# --------------------------------------------
#                   COMPILE
# --------------------------------------------
# Config kernel
kernel_path='./kernels/test_asm/'
kernel_number = 1 # Kernel number (from 1 to 15)
column_usage = [True, False] # Columns to use
nInstrPerCol = 13 # Number of instructions per column
imem_add_start = 3 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF
version="_v1"
from src import simulator
sim = simulator.SIMULATOR()
sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)
sim.compileAsmToHex(kernel_path, kernel_number, version=version)

Processing file: ./kernels/test_asm/instructions_asm_v1.csv...


In [7]:
kernel_path='./kernels/test_asm/'
kernel_number = 2 # Kernel number (from 1 to 15)
column_usage = [True, True] # Columns to use
nInstrPerCol = 4 # Number of instructions per column
imem_add_start = 30 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF
version="_v2"
#sim = simulator.SIMULATOR() # Coment this to store two kernels at the same time in the imem
# Using two cols
sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)
sim.compileAsmToHex(kernel_path, kernel_number, version=version)

Processing file: ./kernels/test_asm/instructions_asm_v2.csv...


In [8]:
# --------------------------------------------
#                   SIMULATE
# --------------------------------------------
from src.simulator import *
from random import randint

sim = SIMULATOR()

# --------------------------------------------
#               KERNEL CONFIGURATION
# --------------------------------------------
kernel_path = './kernels/test_asm/' # Path
kernel_number = 1 # Kernel number (from 1 to 15)
column_usage = [True, False] # Columns to use
nInstrPerCol = 11 # Number of instructions per column
imem_add_start = 0 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF
version="_v1"

sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)

# --------------------------------------------
#                LOAD SPM DATA
# --------------------------------------------
# Load vector by vector in the positions you want
random_vector = [randint(0, 100) for _ in range(SPM_NWORDS)]
nline = 3
sim.setSPMLine(nline, random_vector)
# Or, load the entire content of the SPM
data = []
for nline in range(SPM_NLINES):
    random_vector = [randint(0, 100) for _ in range(SPM_NWORDS)]
    data.append(random_vector)
sim.loadSPMData(data)

# --------------------------------------------
#              COMPILE ASM TO HEX
# --------------------------------------------
# If hexadecimal is not provided, here we generate it
sim.compileAsmToHex(kernel_path, kernel_number, version=version)

# --------------------------------------------
#                 LOAD KERNEL
# --------------------------------------------

# This needs the hex instructions, if you don't provide them, generate then compiling the asm
sim.kernel_load(kernel_path, version=version, kernel_number=kernel_number)

# --------------------------------------------
#               SIMULATE EXECUTION
# --------------------------------------------
show_lcu = []
show_srf = []
show_lsu = []
show_rcs = [[],[],[],[]]
show_mxcu = []
display_ops = [show_lcu, show_lsu, show_mxcu, show_rcs, show_srf]

sim.displaySPMLine(3)

sim.run(kernel_number, display_ops=display_ops)

Processing file: ./kernels/test_asm/instructions_asm_v1.csv...
Processing file: ./kernels/test_asm/instructions_hex_v1.csv...
SPM 3: [64, 69, 58, 1, 55, 65, 92, 5, 18, 22, 81, 17, 57, 15, 13, 36, 43, 100, 54, 36, 21, 71, 35, 11, 88, 60, 98, 12, 84, 36, 28, 9, 89, 7, 61, 79, 59, 90, 16, 1, 15, 53, 30, 35, 58, 92, 78, 78, 63, 48, 4, 31, 86, 98, 93, 71, 39, 96, 86, 7, 35, 82, 69, 25, 54, 30, 28, 75, 4, 38, 60, 35, 57, 66, 98, 39, 57, 35, 40, 54, 17, 82, 89, 83, 79, 86, 82, 57, 27, 44, 82, 92, 46, 4, 99, 46, 53, 7, 41, 43, 0, 61, 73, 9, 58, 49, 5, 36, 93, 33, 44, 36, 64, 79, 90, 2, 15, 35, 34, 86, 41, 87, 58, 22, 54, 74, 73, 36, ]
---------------------------------------------
                PC: 0
---------------------------------------------
------- Col: 0 -------
MXCU: SADD R1, ONE, LAST (selected: 0, not writting SRF)
LSU: BITREV R0, ONE, ONE/LD.VWR VWR_A --> 0
RC: NOP --> 0
RC: NOP --> 0
RC: NOP --> 0
RC: NOP --> 0
LCU: SADD R1, ONE, ZERO --> 1
-----------------------------------------

In [9]:
kernel_path = './kernels/exit/' 
sim.compileHexToAsm(kernel_path)

Processing file: ./kernels/exit/instructions_hex.csv...


In [10]:
kernel_path = './kernels/add_vectors/' 
sim.compileHexToAsm(kernel_path)

Processing file: ./kernels/add_vectors/instructions_hex.csv...


In [11]:
# --------------------------------------------
#               KERNEL CONFIGURATION
# --------------------------------------------
kernel_path = './kernels/add_vectors/' 
kernel_number = 1 # Kernel number (from 1 to 15)
column_usage = [True, False] # Columns to use
nInstrPerCol = 37 # Number of instructions per column
imem_add_start = 0 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF
version=""
sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)
sim.compileAsmToHex(kernel_path, kernel_number, version=version)

Processing file: ./kernels/add_vectors/instructions_asm.csv...


In [12]:

# --------------------------------------------
#               KERNEL CONFIGURATION
# --------------------------------------------
kernel_path = './kernels/mf_q64_erosion/' 
kernel_number = 1 # Kernel number (from 1 to 15)
column_usage = [True, False] # Columns to use
nInstrPerCol = 44 # Number of instructions per column
imem_add_start = 0 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF
version=""
sim = simulator.SIMULATOR()
sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)
sim.compileHexToAsm(kernel_path)
sim.compileAsmToHex(kernel_path, kernel_number, version=version)


Processing file: ./kernels/mf_q64_erosion/instructions_hex.csv...
Processing file: ./kernels/mf_q64_erosion/instructions_asm.csv...


ValueError: Instructions not valid for this cycle (5) of the CGRA. Only the RC on row 0 can write to the SRF.

In [ ]:
from src import *
# Load an existing imem word and decode it
imem_pos = 1
imem_word = 0xe923

rc_imem = RC_IMEM()
rc_imem.set_word(imem_word, imem_pos)
asm = rc_imem.get_instr_pseudo_asm(imem_pos)
print("Pseudo-Assembly representation: " + asm)
rc_imem.get_instruction_info(imem_pos)

# Load an existing imem word and decode it
imem_pos = 1
imem_word = 0x2a920

rc_imem = RC_IMEM()
rc_imem.set_word(imem_word, imem_pos)
asm = rc_imem.get_instr_pseudo_asm(imem_pos)
print("Pseudo-Assembly representation: " + asm)
rc_imem.get_instruction_info(imem_pos)

Pseudo-Assembly representation: LOR R1, SRF(X), ZERO
Performing ALU operation LOR between operands SRF and ZERO
ALU is performing operations with 32-bit precision
Writing ALU result to RC register 1
Pseudo-Assembly representation: LOR SRF(X), ZERO, ZERO
Performing ALU operation LOR between operands ZERO and ZERO
ALU is performing operations with 32-bit precision
No RC registers are being written
